# Generic optimization problem

## Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# optimization passes
from quadratic_program import QuadraticProgram
from workflows import QuadraticConverter, QuadraticPostprocess

# translations
from translators import docplex_mp_to_qp
from translators import qubo_to_sparse_pauli_op

# Docplex - classical description of optimization problems
from docplex.mp.model import Model

# SPSA
from spsa import minimize_spsa

## Load the IBM Quantum service (if using)

In [2]:
#service = QiskitRuntimeService()

In [3]:
#backend = service.get_backend('ibm_nazca')

## **Step 1**: Problem specification using standard tools and conversion to Quantum native format

Specify optimization problem using docplex and convert to Quadratic Unconstained Binary Opimization (QUBO) problem that can be cast as an Ising Hamiltonian suitable for a quantum solution.

In [4]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x*y - x*z + 2*y*z + x - 2*y + 3*z)
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



### Convert to our `QuadraticProgram` format

In [5]:
qp = docplex_mp_to_qp(mdl)
print(qp.prettyprint())

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y + 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



### Classical transformation to QUBO problem and Ising Hamiltonian

In [6]:
quadratic_transformer = QuadraticConverter()
qubo = quadratic_transformer.run(qp)
print(qubo.prettyprint())

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y + 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



In [7]:
hamiltonian, offset = qubo_to_sparse_pauli_op(qubo)
hamiltonian

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[-0.5 +0.j,  0.25+0.j, -1.75+0.j,  0.25+0.j, -0.25+0.j,  0.5 +0.j])

## **Step 2**: Setup quantum solution method and optimize

### Define ansatz circuit from circuit library

In [8]:
ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=2)

## **Step 3**: Solve using quantum primitives

### Standard cost function definition

In [9]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [10]:
#session = Session(backend=backend)
#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

### Perform minimization using classical SPSA optimizer

In [11]:
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=100)
res

 message: Optimization terminated successfully.
 success: True
     fun: -3.5006404750158655
       x: [ 7.263e+00  7.862e+00  1.472e+00  2.173e+00  7.657e+00
            3.705e+00 -3.142e+00  3.361e+00  7.290e+00]
     nit: 100
    nfev: 200

### Computute sampled distribution at optimal values

In [12]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

## **Step 4**: Post-process and return answer in classical format
Transform quantum solution and convert back into classical variable space

In [13]:
solution =QuadraticPostprocess(qubo, quadratic_transformer).run(samp_dist)
solution

array([0., 1., 0.])